In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# imports
import numpy as np 
import matplotlib.pyplot as plt
import pickle

# Doc2Vec applications

In [15]:
# before importin you need to install it
import gensim 

In [23]:
import os
os.chdir(r'/home/ilaria/Documenti/gensim/gensim')
import gensim.models as g

## Pre-processing and file reading

In [3]:
os.chdir(r'/home/ilaria/Scrivania/Machine_Learning/Project_2/Project2/data/twitter_datasets_epfl/')

from pickle_vocab import main as pickle_vocab
from cooc import main as cooc
from glove_solution import main as glove


In [19]:
os.chdir(r'/home/ilaria/Scrivania/Machine_Learning/Project_2/Project2/data/twitter_datasets_epfl/unique/short/')

In [5]:
%%bash

chmod u+x ./build_vocab.sh
./build_vocab.sh
chmod u+x ./cut_vocab.sh
./cut_vocab.sh

In [6]:
pickle_vocab()
cooc()

pickle_vocab.py executed
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
summing duplicates ... (this can take a while)
summing duplicates FINISHED !


# GloVe 

In [7]:
glove()

loading cooccurrence matrix
6361648 nonzero entries
using nmax = 100 , cooc.max() = 207302
initializing embeddings
epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [8]:
X = np.load('embeddings.npy')
print(X.shape)

(21710, 25)


In [9]:
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [26]:
import multiprocessing
# set doc2vec parameters
vector_size = 100
window_size = 10
min_count = 1
sampling_threshold = 1e-5
negative_size = 5
train_epoch = 100
dm = 0 #0 = dbow; 1 = dmpv

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"


In [27]:
# Input with positive

train_corpus_pos = 'train_pos.txt'

# output model 

save_path = 'model.bin'

#train doc2vec model
docs = g.doc2vec.TaggedLineDocument(train_corpus_pos)
model = g.Doc2Vec(docs, size=vector_size, window=window_size, min_count=min_count, sample=sampling_threshold, workers=cores, hs=0, dm=dm, negative=negative_size, dbow_words=1, dm_concat=1, pretrained_emb=pretrained_emb, iter=train_epoch)


NameError: name 'pretrained_emb' is not defined

# Application of Doc2Vec with the vocab built with GloVe

# Application of PVDM

# Word2Vec + Doc2Vec